Obtención de los datos

In [33]:
import pandas as pd

# Cargar datos
path='C:/4_F5/011_proyecto_ml/'


df = pd.read_csv(path + '/airline_passenger_satisfaction.csv')

#Se elimina la primera columna que era el subíndice del csv
df = df.iloc[:, 1:] 
df=df.drop(columns='id') #no sé si al final quito el id porque me estorba

In [34]:
#Hay que crear una parte que sea 'test'
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=22 )

# test_size=0.2, este parámetro indica que deseas que el 20% de los datos se utilicen como conjunto de prueba y el 80% como conjunto de entrenamiento. 
# random_state=22 Este parámetro establece una semilla para el generador de números aleatorios que se usa para realizar la división. Esto asegura que cada vez que ejecutes el código, obtendrás la misma división de los datos en conjuntos de entrenamiento y prueba. Usar un valor de random_state fijo es una buena práctica cuando quieres que tus resultados sean reproducibles.

train.to_csv(path+'df_train.csv', index=False)
test.to_csv(path+'df_test.csv', index=False)
#Este test no lo tocamos hasta el final

3. Exploración y visualización de Datos

In [35]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Gender                             103904 non-null  object 
 1   Customer Type                      103904 non-null  object 
 2   Age                                103904 non-null  int64  
 3   Type of Travel                     103904 non-null  object 
 4   Class                              103904 non-null  object 
 5   Flight Distance                    103904 non-null  int64  
 6   Inflight wifi service              103904 non-null  int64  
 7   Departure/Arrival time convenient  103904 non-null  int64  
 8   Ease of Online booking             103904 non-null  int64  
 9   Gate location                      103904 non-null  int64  
 10  Food and drink                     103904 non-null  int64  
 11  Online boarding                    1039

In [36]:
print(f'\n\nEl DF tiene {df.shape[0]:,} filas')
print(f'El DF tiene {df.shape[1]} columnas')



El DF tiene 103,904 filas
El DF tiene 23 columnas


In [37]:
# Comprobar cuántos valores missing existen en el DataFrame por variable

nulos={'conteo_nulos':df.isnull().sum(),'proporcion%':round(df.isnull().sum()/df.shape[0]*100, 3)}

df_nulos=pd.DataFrame(data=nulos)
print(df_nulos)

                                   conteo_nulos  proporcion%
Gender                                        0        0.000
Customer Type                                 0        0.000
Age                                           0        0.000
Type of Travel                                0        0.000
Class                                         0        0.000
Flight Distance                               0        0.000
Inflight wifi service                         0        0.000
Departure/Arrival time convenient             0        0.000
Ease of Online booking                        0        0.000
Gate location                                 0        0.000
Food and drink                                0        0.000
Online boarding                               0        0.000
Seat comfort                                  0        0.000
Inflight entertainment                        0        0.000
On-board service                              0        0.000
Leg room service        

In [38]:
# Conteo de los valores por cada columna ----> Hacer bucle para ver si hay columnas que estén desbalanceadas.
# Para ver qué variables hay que escalar

df['Customer Type'].value_counts()

Customer Type
Loyal Customer       84923
disloyal Customer    18981
Name: count, dtype: int64

Antes de convertir las variables categóricas, hay que tratar los 'missings'.

Que según hemos concluido, tantos los valores '0' como los 'espacios en blanco' son 'missing' y los vamos a eliminar.

In [39]:
# Primero elimino los 'missings'
df = df.dropna(how='any')

print(f'\n\nEl DF tiene {df.shape[0]:,} filas')


# Compruebo que se ha eliminado
nulos={'conteo_nulos':df.isnull().sum(),'proporcion%':round(df.isnull().sum()/df.shape[0]*100, 3)}
df_nulos=pd.DataFrame(data=nulos)
print(df_nulos)



El DF tiene 103,594 filas
                                   conteo_nulos  proporcion%
Gender                                        0          0.0
Customer Type                                 0          0.0
Age                                           0          0.0
Type of Travel                                0          0.0
Class                                         0          0.0
Flight Distance                               0          0.0
Inflight wifi service                         0          0.0
Departure/Arrival time convenient             0          0.0
Ease of Online booking                        0          0.0
Gate location                                 0          0.0
Food and drink                                0          0.0
Online boarding                               0          0.0
Seat comfort                                  0          0.0
Inflight entertainment                        0          0.0
On-board service                              0          

In [40]:
#Ahora elimino los '0' pero de las columnas categóricas ordinales



In [41]:
import pandas as pd
from pandas.api.types import is_categorical_dtype

# Función para detectar variables categóricas ordinales
def detectar_variables_ordinales(df):
    ordinal_columns = []

    for column in df.columns:
        # verificar si la columna es numérica
        if pd.api.types.is_numeric_dtype(df[column]):
            # obtener valores únicos
            unique_values = df[column].unique()
            # verificar si todos los valores están en el rango [1,5]
            if set(unique_values).issubset({0, 1, 2, 3, 4, 5}):
                ordinal_columns.append(column)

    
    return ordinal_columns




variables_ordinales = detectar_variables_ordinales(df)
print("Variables categóricas ordinales:", variables_ordinales)
rows_with_zero = (df[variables_ordinales] == 0).any(axis=1).sum()
print(rows_with_zero)
# Resultado:
# Variables categóricas ordinales: ['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']
# De esas variables hay que quitar los '0' porque hemos decidido que son missing. Antes de hacer el one-hot-encoding


Variables categóricas ordinales: ['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']
8179


In [42]:
""" count_zeros = (df == 0).sum().sum()
print(count_zeros) """

# Contar los valores iguales a 0 en las columnas ordinales
conteo_ceros = (df[variables_ordinales] == 0).sum()   ## conteo_ceros = df[variables_ordinales].apply(lambda x: (x == 0).sum())

# Sumar el conteo para obtener el total de ceros en todas las columnas
total_ceros = conteo_ceros.sum()

# Mostrar
print("\nConteo de '0's por columna:\n\n", conteo_ceros)
print(f"\nTotal de '0's en todas las columnas: {total_ceros:,}")



Conteo de '0's por columna:

 Inflight wifi service                3096
Departure/Arrival time convenient    5290
Ease of Online booking               4473
Gate location                           1
Food and drink                        105
Online boarding                      2420
Seat comfort                            1
Inflight entertainment                 14
On-board service                        3
Leg room service                      470
Baggage handling                        0
Checkin service                         1
Inflight service                        3
Cleanliness                            12
dtype: int64

Total de '0's en todas las columnas: 15,889


In [43]:
df_sin_ceros = df[(df[variables_ordinales] != 0).all(axis=1)]#.reset_index()

In [44]:
""" for col in variables_ordinales:
    df_sin_ceros = df[df[col] != 0] #df_sin_ceros = df[(df != 0).all(axis=1)]
    
     """
    
    
#Vuelto a contar los '0's en el dataframe de las variables ordinales para comprobar que se han eliminado
conteo_ceros = (df_sin_ceros[variables_ordinales] == 0).sum()   ## conteo_ceros = df[variables_ordinales].apply(lambda x: (x == 0).sum())

#¿Porque predice mal? o porque está sesgado (es inherente a la muestra tomada)
# Aleatorio no es lo mismo que representativa

# Sumar el conteo para obtener el total de ceros en todas las columnas
total_ceros = conteo_ceros.sum()

# Mostrar
print("\nConteo de ceros por columna:\n\n", conteo_ceros)
print("\nTotal de ceros en todas las columnas:", total_ceros)

print(f'Quedan {df_sin_ceros.shape[0]:,} filas')
# Contar las filas que contienen algún 0 en las columnas especificadas
rows_with_zero = (df[variables_ordinales] == 0).any(axis=1).sum()

print(f"Número de filas que tenían algún '0' en las columnas ordinales: {rows_with_zero}")


Conteo de ceros por columna:

 Inflight wifi service                0
Departure/Arrival time convenient    0
Ease of Online booking               0
Gate location                        0
Food and drink                       0
Online boarding                      0
Seat comfort                         0
Inflight entertainment               0
On-board service                     0
Leg room service                     0
Baggage handling                     0
Checkin service                      0
Inflight service                     0
Cleanliness                          0
dtype: int64

Total de ceros en todas las columnas: 0
Quedan 95,415 filas
Número de filas que tenían algún '0' en las columnas ordinales: 8179


Antes de hacer la matriz de correlación, hay que convertir las columnas que son 'object' a binarias.

(pero cuidado, PRIMERO hay que separar la variable objetivo, que es una columna del dataframe)

**One-Hot Encoding es generalmente preferido cuando no hay un orden inherente en las categorías. De esta manera, me aseguro de que el modelo no asuma relaciones incorrectas entre las categorías.

**Label Encoding podría usarse si hay un orden claro entre las categorías

In [45]:
import category_encoders as ce
from sklearn.preprocessing import OrdinalEncoder
# Separar la variable objetivo antes de la codificación, la llamo 'y'.
# Si no la separo todavía, la va a codificar, lo cual no sé si conviene por el momento.
y = df_sin_ceros['satisfaction']

#-- Codificar solo las variables categóricas--

# Variables categóricas
encoder_cat = ce.OneHotEncoder(use_cat_names=True)
x_encoded = encoder_cat.fit_transform(df_sin_ceros.drop(columns=['satisfaction']))
print(x_encoded)


# Variables categóricas ordinales
# Crear OrdinalEncoder
encoder_ord = OrdinalEncoder()
# Aplicar OrdinalEncoder a múltiples variables categóricas ordinales.
# Machaco la variable x_encoded que la recojo de la codificación anterior.
x_encoded  = pd.DataFrame(encoder_ord.fit_transform(x_encoded), columns=x_encoded.columns)



# Concatenar la columna 'satisfaccion' de nuevo al DataFrame codificado
df_encoded = pd.concat([x_encoded, y], axis=1)

""" # Dividir los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x_encoded, y, test_size=0.2, random_state=42)

# Guardar datos en ficheros
path='C:/4_F5/011_proyecto_ml/'
x_train.to_csv(path + '/x_train.csv')
x_test.to_csv(path + '/x_test.csv')
y_train.to_csv(path + '/y_train.csv')
y_test.to_csv(path + '/y_test.csv')
 """



        Gender_Male  Gender_Female  Customer Type_Loyal Customer  \
0                 1              0                             1   
1                 1              0                             0   
2                 0              1                             1   
3                 0              1                             1   
4                 1              0                             1   
...             ...            ...                           ...   
103899            0              1                             0   
103900            1              0                             1   
103901            1              0                             0   
103902            0              1                             0   
103903            1              0                             1   

        Customer Type_disloyal Customer  Age  Type of Travel_Personal Travel  \
0                                     0   13                               1   
1                      

" # Dividir los datos en conjuntos de entrenamiento y prueba\nx_train, x_test, y_train, y_test = train_test_split(x_encoded, y, test_size=0.2, random_state=42)\n\n# Guardar datos en ficheros\npath='C:/4_F5/011_proyecto_ml/'\nx_train.to_csv(path + '/x_train.csv')\nx_test.to_csv(path + '/x_test.csv')\ny_train.to_csv(path + '/y_train.csv')\ny_test.to_csv(path + '/y_test.csv')\n "


Cuando tienes un DataFrame con columnas categóricas y categóricas ordinales, es importante elegir el método de codificación adecuado que preserve la naturaleza ordinal de la columna.

In [46]:
import janitor  # Esto importa pyjanitor y extiende pandas

Voy a separar las variables, por un lado según sean categóricas(binarias, ordinales..) y por otro las numéricas

· Categóricas

· Numéricas

Porque a veces un modelo va a trabajar mejor sólo con las categóricas o sólo con las numéricas.
Además, es conveniente escalar las variables munéricas (ya sean por cambios de escala o por magnitud, o incluso 'normalizarlas')

In [47]:
#Filtro para quedarme con las numéricas, las que voy a escalar

# Primero detecto las columnas que tienen solo valores de 0 y 1
binary_cols = x_encoded.columns[x_encoded.nunique() == 2]

# Luego detecto las columnas ordinales (2 a 5)
ordinal_cols = x_encoded.columns[(x_encoded.nunique() > 2) & (x_encoded.nunique() <= 5)]

# Ambas categóricas las junto 
categorical_cols = list(binary_cols) + list(ordinal_cols)



¿Por qué este orden es importante?

Consistencia: Al separar primero X e y y luego hacer el split, garantizas que las filas correspondientes de X e y coincidan exactamente en los conjuntos de entrenamiento y prueba.

Evitar filtraciones de datos: Si haces el split antes de separar X e y, podrías accidentalmente introducir fugas de datos o inconsistencias, lo que podría afectar la capacidad del modelo para generalizar.

In [48]:
#Primero separas la 'y' de las variables predictoras 'x'
#Luego El split de tanto 'y' como 'x' con su correspondiente 'train' y 'test' se hace al final, cuando ya están las variables numéricas escaladas y realizado el coding y lebel en las categóricas
#Finalmente voy a escalar las numéricas: Después de realizar el split, ajustas el escalador (Scaler) solo con el conjunto de entrenamiento (X_train). Esto asegura que las estadísticas de escalado (media, desviación estándar, etc.) no estén influenciadas por los datos de prueba.
#Aplicar la transformación de escalado al conjunto de prueba:Una vez que el escalador se ha ajustado al conjunto de entrenamiento, usas ese mismo escalador para transformar el conjunto de prueba (X_test). Esto asegura que ambos conjuntos sean escalados de manera consistente
from sklearn.preprocessing import StandardScaler


print(f'Las variables categóricas son:\n' + '\n'.join(categorical_cols) + '\n')


# Separa el dataframe en partes de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x_encoded, y, test_size=0.2, random_state=42)



# Filtro para quitar las columnas categóricas del dataframe y quedarme con las numéricas, porque son esas las que voy a escalar
x_num_train = x_train.drop(columns=categorical_cols).columns #Aquí Sólo se están guardando el nombre de las columnas para rastrearlas
pd.DataFrame(x_train[x_num_train]).to_csv(path+'/x_num_train_no_scaled.csv', index=False)

x_num_test = x_test.drop(columns=categorical_cols).columns #Sólo cojo el nombre de las columnas para rastrearlas
pd.DataFrame(x_test[x_num_test]).to_csv(path+'/x_num_test_no_scaled.csv', index=False)


print(f'Las variables numéricas son:\n' + '\n'.join(x_num_train) + '\n')

# Crea un objeto StandardScaler y ajusta los parámetros de escalado con la parte de entrenamiento
scaler = StandardScaler()
x_train_num_scaled = scaler.fit_transform(x_train[x_num_train])

# Aplica los parámetros de escalado aprendidos a la parte de prueba
x_test_num_scaled = scaler.transform(x_test[x_num_test]) #devuelve un array de arrays



Las variables categóricas son:
Gender_Male
Gender_Female
Customer Type_Loyal Customer
Customer Type_disloyal Customer
Type of Travel_Personal Travel
Type of Travel_Business travel
Class_Eco Plus
Class_Business
Class_Eco
Inflight wifi service
Departure/Arrival time convenient
Ease of Online booking
Gate location
Food and drink
Online boarding
Seat comfort
Inflight entertainment
On-board service
Leg room service
Baggage handling
Checkin service
Inflight service
Cleanliness

Las variables numéricas son:
Age
Flight Distance
Departure Delay in Minutes
Arrival Delay in Minutes



La razón es que, en general, deseas mantener la separación entre los datos de entrenamiento y los datos de prueba, incluso después de la escalado. De esta manera, puedes asegurarte de que el modelo de machine learning se entrena y se evalúa correctamente.

In [49]:

# Reemplazo en 'train' las columnas numéricas sin escalar con las columnas numéricas escaladas. Por lo que ya me queda todo 'x' junto y escalado
x_train[x_num_train] = x_train_num_scaled 
print(x_train)

x_train.to_csv(path +   '/x_train_scaled.csv')

# Reemplazo en 'test' las columnas numéricas sin escalar con las columnas numéricas escaladas
x_test[x_num_test] = x_test_num_scaled
x_test.to_csv(path + '/x_test_scaled.csv')

print(x_test)

       Gender_Male  Gender_Female  Customer Type_Loyal Customer  \
34944          0.0            1.0                           1.0   
2549           1.0            0.0                           1.0   
90416          0.0            1.0                           1.0   
68941          0.0            1.0                           1.0   
32018          1.0            0.0                           1.0   
...            ...            ...                           ...   
6265           1.0            0.0                           1.0   
54886          0.0            1.0                           1.0   
76820          1.0            0.0                           1.0   
860            0.0            1.0                           0.0   
15795          0.0            1.0                           1.0   

       Customer Type_disloyal Customer       Age  \
34944                              0.0  1.277549   
2549                               0.0  0.545219   
90416                              0.0 

In [50]:
# Guardar datos en ficheros
""" 
path='C:/4_F5/011_proyecto_ml/'
pd.DataFrame(x_train_scaled).to_csv('x_train_scaled.csv', index=False)
pd.DataFrame(x_test_scaled).to_csv('x_test_scaled.csv', index=False)
y_train.to_csv(path + '/y_train.csv')
y_test.to_csv(path + '/y_test.csv')

 """

" \npath='C:/4_F5/011_proyecto_ml/'\npd.DataFrame(x_train_scaled).to_csv('x_train_scaled.csv', index=False)\npd.DataFrame(x_test_scaled).to_csv('x_test_scaled.csv', index=False)\ny_train.to_csv(path + '/y_train.csv')\ny_test.to_csv(path + '/y_test.csv')\n\n "

In [51]:
import matplotlib.pyplot as plt

# Histograma de la variable escalada
plt.hist(x_test['Age'], bins=50)
plt.title('Distribución de la variable escalada')
plt.show()

# Histograma de la variable no escalada ---> usar de A´ngel


In [52]:
print('Estadísticas descriptivas de la variable escalada:')
print(x_test['Age'].describe())

print('Estadísticas descriptivas de la variable escalada:')
#print(x_test_scaled[:, 0]).describe()) 

Estadísticas descriptivas de la variable escalada:
count    19083.000000
mean         0.000552
std          1.000656
min         -2.184373
25%         -0.786289
50%          0.012616
75%          0.744946
max          2.742208
Name: Age, dtype: float64
Estadísticas descriptivas de la variable escalada:


Primer modelo

In [53]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Crea un objeto RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrena el modelo con el conjunto de entrenamiento
model.fit(x_train, y_train)

# Evalúa el modelo con el conjunto de prueba
y_pred = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9644185924644972


In [54]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# ...

y_pred = model.predict(x_test)

# Calcula la accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


# Calcula la precisión (average='weighted' for multi-class problems)
precision = precision_score(y_test, y_pred, average='weighted')
print("Precision:", precision)

# Calcula el recall
recall = recall_score(y_test, y_pred, average='weighted',labels=['neutral or dissatisfied', 'satisfied'])
print("Recall:", recall)

""" # Calcula el F1-score
f1 = f1_score(y_test, y_pred)
print("F1-score:", f1)

# Calcula el ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred)
print("ROC-AUC:", roc_auc)

# Calcula la matriz de confusión
conf_mat = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_mat) """

Accuracy: 0.9644185924644972
Precision: 0.9646400001026753
Recall: 0.9644185924644972


' # Calcula el F1-score\nf1 = f1_score(y_test, y_pred)\nprint("F1-score:", f1)\n\n# Calcula el ROC-AUC\nroc_auc = roc_auc_score(y_test, y_pred)\nprint("ROC-AUC:", roc_auc)\n\n# Calcula la matriz de confusión\nconf_mat = confusion_matrix(y_test, y_pred)\nprint("Confusion Matrix:")\nprint(conf_mat) '

Multicolinealidad

Probando 

In [55]:
""" 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import numpy as np


le = LabelEncoder()
y_train_le = le.fit_transform(y_train)

lin_reg = LinearRegression()
lin_reg.fit(x_train,y_train_le)


#print(x_train.head())
print(y_train.head())
print(lin_reg.predict(x_train.head()))

"""



""" 
El problema es que el modelo de regresión lineal (LinearRegression) no entiende que tu variable objetivo es binaria.

La regresión lineal es un algoritmo de aprendizaje automático que se utiliza para predecir valores continuos, no binarios. 
Cuando entrenas el modelo con tus datos, el algoritmo intenta encontrar la mejor línea que se ajuste a tus datos, pero no 
tiene en cuenta que la variable objetivo es binaria.

Por lo tanto, cuando haces print(lin_reg.predict(x_train.head())), el modelo devuelve valores decimales porque está tratando de predecir un valor continuo, no un valor binario.

Para solucionar este problema, debes utilizar un algoritmo de aprendizaje automático que sea adecuado para problemas de clasificación binaria, como LogisticRegression de scikit-learn. Este algoritmo entiende que la variable objetivo es binaria y devuelve probabilidades de pertenencia a cada clase, que puedes convertir en valores binarios (0 o 1) utilizando una función de umbral (threshold).

 """

' \nEl problema es que el modelo de regresión lineal (LinearRegression) no entiende que tu variable objetivo es binaria.\n\nLa regresión lineal es un algoritmo de aprendizaje automático que se utiliza para predecir valores continuos, no binarios. \nCuando entrenas el modelo con tus datos, el algoritmo intenta encontrar la mejor línea que se ajuste a tus datos, pero no \ntiene en cuenta que la variable objetivo es binaria.\n\nPor lo tanto, cuando haces print(lin_reg.predict(x_train.head())), el modelo devuelve valores decimales porque está tratando de predecir un valor continuo, no un valor binario.\n\nPara solucionar este problema, debes utilizar un algoritmo de aprendizaje automático que sea adecuado para problemas de clasificación binaria, como LogisticRegression de scikit-learn. Este algoritmo entiende que la variable objetivo es binaria y devuelve probabilidades de pertenencia a cada clase, que puedes convertir en valores binarios (0 o 1) utilizando una función de umbral (threshold

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Codificaicón de la variable objetivo
le = LabelEncoder()
y_train_le = le.fit_transform(y_train)


# Entrenamiento del modelo
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train_le)


# Predicción en el conjunto de prueba
y_test_le = le.transform(y_test)  # Codifica y_test para compararlo correctamente
y_pred = log_reg.predict(x_test)  # Predicción sobre el conjunto de prueba

# Evaluación del modelo
accuracy = accuracy_score(y_test_le, y_pred)
print("Accuracy:", accuracy)

# Ejemplo de predicciones sobre nuevas muestras (primeras 5 filas del conjunto de prueba)
print(x_test.head())
y_pred_example = log_reg.predict(x_test.head())
print(y_pred_example)

Accuracy: 0.8970287690614683
       Gender_Male  Gender_Female  Customer Type_Loyal Customer  \
15480          1.0            0.0                           0.0   
46258          0.0            1.0                           1.0   
89013          0.0            1.0                           1.0   
10012          0.0            1.0                           1.0   
20787          0.0            1.0                           1.0   

       Customer Type_disloyal Customer       Age  \
15480                              1.0 -1.185741   
46258                              0.0  0.744946   
89013                              0.0 -0.919440   
10012                              0.0 -1.585194   
20787                              0.0  0.545219   

       Type of Travel_Personal Travel  Type of Travel_Business travel  \
15480                             0.0                             1.0   
46258                             0.0                             1.0   
89013                             1.

En lugar de calcular la correlación entre las variables dummy, es más útil utilizar otras técnicas para seleccionar las variables más relevantes para el modelo. Algunas opciones son:

Análisis de la importancia de las características: utiliza algoritmos como Random Forest o Gradient Boosting para evaluar la importancia de cada variable en la predicción de Y.
Selección de características: utiliza técnicas como la selección recursiva de características (RFE) o la selección de características mediante la matriz de correlación de mutual information.
Análisis de la varianza: utiliza técnicas como la descomposición de la varianza para evaluar la contribución de cada variable a la varianza de Y.
Recuerda que la selección de variables es un paso importante en el proceso de modelado, y es importante utilizar técnicas adecuadas para identificar las variables más relevantes para el modelo.

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Codificación de la variable objetivo
le = LabelEncoder()
y_train_le = le.fit_transform(y_train)

# Entrenamiento del modelo
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train_le)

# Predicción en el conjunto de prueba
y_test_le = le.transform(y_test)  # Codifica y_test para compararlo correctamente
y_pred = log_reg.predict(x_test)  # Predicción sobre el conjunto de prueba

# Evaluación del modelo
accuracy = accuracy_score(y_test_le, y_pred)
print("Accuracy:", accuracy)

# Ejemplo de predicciones sobre nuevas muestras (primeras 5 filas del conjunto de prueba)
print("Primeras 5 filas de x_test:")
print(x_test.head())

# Predicciones para las primeras 5 filas del conjunto de prueba
y_pred_example = log_reg.predict(x_test.head())
print("\nPredicciones para las primeras 5 filas:")
print(y_pred_example)

# Convertir las predicciones numéricas a etiquetas originales
y_pred_example_labels = le.inverse_transform(y_pred_example)
print("\nPredicciones con etiquetas originales:")
print(y_pred_example_labels)

# Verificar si las predicciones son 'satisfied'
print("\n¿Las predicciones son 'satisfied'?")
print(y_pred_example_labels == 'satisfied')

Accuracy: 0.8970287690614683
Primeras 5 filas de x_test:
       Gender_Male  Gender_Female  Customer Type_Loyal Customer  \
15480          1.0            0.0                           0.0   
46258          0.0            1.0                           1.0   
89013          0.0            1.0                           1.0   
10012          0.0            1.0                           1.0   
20787          0.0            1.0                           1.0   

       Customer Type_disloyal Customer       Age  \
15480                              1.0 -1.185741   
46258                              0.0  0.744946   
89013                              0.0 -0.919440   
10012                              0.0 -1.585194   
20787                              0.0  0.545219   

       Type of Travel_Personal Travel  Type of Travel_Business travel  \
15480                             0.0                             1.0   
46258                             0.0                             1.0   
89013   

4. Preparación de los Algoritmos de ML

5. Seleccionar un modelo y entrénalo

6. Ajustar el modelo (optimización con hypermaprámetros, ensamblado)

7. Presentar la solución.

8. Desplegar y monitorizar